In [1]:
import time
import pymongo
import re
import json
import requests
import pandas as pd

from tqdm import tqdm
from pprint import pprint
from pathlib import Path
from bs4 import BeautifulSoup

#import local libraries
import lib_sepinpho as sep
import lib_fileops as io

##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#connect to database
db = client.visualizing_sep

In [ ]:
#export list of all articles with domain_tags and primary_domain
# commented out to not accidently run again

# collection_to_update = db.sep_entries
# all_domains = pd.DataFrame(collection_to_update.find({},
#                                               {'title':1, 'page_url', 'author': 1, 'primary_domain':1, 'domain_tags':1}))
# all_domains.to_csv('domains.csv')

In [2]:
#import updated domain data
# commented out to not accidently run again

collection_to_update = db.sep_entries


# updated_domains = pd.read_csv('domains.csv')

# for index,row in tqdm(updated_domains.iterrows()):
#     title = row['title']
#     domain_tags = row['domain_tags']
#     primary_domain = row['primary_domain']
#     # update function to include page_url
#     sep.update_domain_info(title,domain_tags, primary_domain, collection_to_update)

In [8]:
#get distinct list of primary_domains
db.sep_entries.distinct('primary_domain')

['Aesthetics and Philosophy of Art',
 'African and African-American Philosophy',
 'Arabic and Islamic Philosophy',
 'Chinese Philosophy',
 'Continental Philosophy',
 'Epistemology',
 'Ethics',
 'Feminist Philosophy',
 'Indian Philosophy',
 'Japanese Philosophy',
 'Latin American and Iberian Philosophy',
 'Logic',
 'Metaphysics',
 'Philosophy of Action',
 'Philosophy of Biology',
 'Philosophy of Computer Science',
 'Philosophy of Economics',
 'Philosophy of Language',
 'Philosophy of Law',
 'Philosophy of Mathematics',
 'Philosophy of Mind',
 'Philosophy of Physics',
 'Philosophy of Religion',
 'Philosophy of Science',
 'Social and Political Philosophy',
 'Thinker']

In [9]:
#extract unique list of domain tags
domain_tags = list(db.sep_entries.find( filter={},
                                     projection={'domain_tags':1, '_id':0}))
domain_tags_individual = []

for tag in domain_tags:
    semisplit_tags = tag['domain_tags'].split(';')
    for semisplit_tag in semisplit_tags:
        commasplit_tags = semisplit_tag.split(',')
        for commasplit_tag in commasplit_tags:
            if commasplit_tag != '':
                domain_tags_individual.append(commasplit_tag.strip())

individual_tags = sorted(set(domain_tags_individual))
pprint(individual_tags)

['Aesthetics and Philosophy of Art',
 'African and African-American Philosophy',
 'Arabic and Islamic Philosophy',
 'Chinese Philosophy',
 'Continental Philosophy',
 'Epistemology',
 'Ethics',
 'Feminist Philosophy',
 'Indian Philosophy',
 'Japanese Philosophy',
 'Judaic Philosophy',
 'Latin American and Iberian Philosophy',
 'Logic',
 'Metaphysics',
 'Philosophy of Action',
 'Philosophy of Biology',
 'Philosophy of Computer Science',
 'Philosophy of Economics',
 'Philosophy of Language',
 'Philosophy of Law',
 'Philosophy of Mathematics',
 'Philosophy of Mind',
 'Philosophy of Physics',
 'Philosophy of Religion',
 'Philosophy of Science',
 'Social and Political Philosophy',
 'Thinker']


In [3]:
# #update db with new domain names
# collection_to_update = db.sep_entries

# search_filter = 'Jewish Philosophy'
# replace_with = 'Judaic Philosophy'

# results = list(collection_to_update.find({'domain_tags': re.compile(search_filter)}, 
#                                          {'page_url': 1, 'title':1, 'domain_tags':1}))

# for result in results:
#     print(result['domain_tags'])
#     new_domain_tags = result['domain_tags'].replace(search_filter, replace_with)
#     print(new_domain_tags)
#     collection_to_update.update_one({'page_url': result['page_url']}, { '$set': {'domain_tags': new_domain_tags}})

# new_results = list(collection_to_update.find({'domain_tags': re.compile(search_filter)}, 
#                                          {'page_url': 1, 'title':1, 'domain_tags':1}))
# pprint(new_results)

Thinker,Philosophy of Religion,Jewish Philosophy
Thinker,Philosophy of Religion,Judaic Philosophy
Thinker,Jewish Philosophy
Thinker,Judaic Philosophy
Thinker,Jewish Philosophy
Thinker,Judaic Philosophy
Thinker, Jewish Philosophy
Thinker, Judaic Philosophy
Arabic and Islamic Philosophy, Jewish Philosophy
Arabic and Islamic Philosophy, Judaic Philosophy
Thinker,Jewish Philosophy,Ethics,Continental Philosophy
Thinker,Judaic Philosophy,Ethics,Continental Philosophy
Thinker, Jewish Philosophy,Epistemology,Philosophy of Religion
Thinker, Judaic Philosophy,Epistemology,Philosophy of Religion
Thinker,Jewish Philosophy
Thinker,Judaic Philosophy
Thinker,Jewish Philosophy,Philosophy of Religion
Thinker,Judaic Philosophy,Philosophy of Religion
Thinker, Jewish Philosophy
Thinker, Judaic Philosophy
Thinker,Jewish Philosophy
Thinker,Judaic Philosophy
Thinker,Jewish Philosophy
Thinker,Judaic Philosophy
Thinker,Jewish Philosophy
Thinker,Judaic Philosophy
Thinker,Philosophy of Religion,Jewish Philosophy

In [10]:
# create text search on preamble_text and main_text
# completed 10.1.2020

collection_to_update = db.sep_entries

collection_to_update.create_index([('preamble_text','text'), ('main_text','text')], name='preamble_textindex')
collection_to_update.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'visualizing_sep.sep_entries'},
 'title_1': {'v': 2,
  'key': [('title', 1)],
  'ns': 'visualizing_sep.sep_entries'},
 'preamble_textindex': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'visualizing_sep.sep_entries',
  'weights': SON([('main_text', 1), ('preamble_text', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [ ]:
#Prior Updates
# African American Philosophy: Completed 10.2.2020
# text_filter = 'African African-American'

#########################################################
# Completed 11.18.2020

# Jewish Philosophy: 
# text_filter = ' "\"Jewish Philosophy\ "\"Hebrew Philosophy\  '

# Aesthetics and Philosophy of Art
# text_filter = ' "\"Jewish Philosophy\ "\"Hebrew Philosophy\  '

# Arabic and Islamic Philosophy
# text_filter = '  "\"Arabic and Islamic Philosophy\ "\"Islamic Philosophy\ "\"Arabic Philosophy\ '

# Chinese Philosophy
# text_filter = '  "\"Chinese Philosophy\ Buddhism Daoism '

# Epistemology
# text_filter = ' Epistemology "\"Theory of Knowedge Truth '

#Feminist Philosophy
# text_filter = ' Feminism "\"Feminist Philosophy\ '

#Indian Philosophy
# text_filter = ' "\"Indian Philosophy\ '

#Japanese Philosophy
# text_filter = ' "\"Japanese Philosophy\ '

#Latin American
# text_filter = ' "\"Latin American Philosophy\ "\"Iberian Philosophy "\"Spanish Philosophy '

#Logic 
# text_filter = ' Logic Inference Entailment'

#Metaphysics
# text_filter = ' Logic Inference Entailment'

#philosopy of biology
# text_filter = ' "\"Philosophy of Biology\ Biology Evolution Genetics Biological Conscoiusness Human Neuroscience'

#philosophy of computer science
# text_filter = ' "\"Computer Science\ "\"Information Technology\ Computation '

#philosophy of economics
# text_filter = ' "\"Philosophy of Economics\ "\"Game Theory\ Economic Markets Capitalism Wealth "\"Decision Theory\ '

#philosophy of language
# text_filter = ' "\"Philosophy of Language\ Linguistics Semantics Pragmatics '

#philosophy of law
# text_filter = ' "\"Philosophy of Law\ Legal Law Testimony'

#philosophy of mathematics
# text_filter = ' "\"Philosophy of Mat\ Legal Law Testimony'

# Philosophy of Mind'
# text_filter = ' "\"Philosophy of Mind\ Mind Consciousness Cognition Emotion Brain Mental Perception Intention'

#  'Philosophy of Physics',
# text_filter = ' "\"Philosophy of Physics\ Physics Relativity Quantum''

#  'Philosophy of Religion',
# text_filter = ' ' "\"Philosophy of Religion\ Theology God Divine Religion''

#  'Philosophy of Science',
# text_filter = '"\"Philosophy of Science\ Scientific  '

#  'Social and Political Philosophy',
# text_filter = ' '
#  'Thinker'
#########################################################

In [2]:
#generate dataframes of domain search results, and then export them to csv for content updates

collection_to_update = db.sep_entries

text_filter = ' "\"Cognitive Science"\" Cognition'
csv_file = 'cognition.csv'
text_results = pd.DataFrame(collection_to_update.find(
                                         {'$text':{'$search': text_filter }},
                                         {'score': { '$meta': 'textScore'}, 
                                            'title':1, 
                                            'page_url':1,
                                            'primary_domain':1, 
                                            'domain_tags':1}
                                         ))

text_results.head(10)                                         
text_score_max =  text_results['score'].max()

text_results_above1 = text_results[text_results.score > text_score_max*0.5]

text_results_above1.to_csv(csv_file)


In [4]:
#update domains from csv 
collection_to_update = db.sep_entries
domains_to_update = pd.read_csv('philaction.csv')

for index,row in tqdm(domains_to_update.iterrows()):
    title = row['title']
    domain_tags = row['domain_tags']
    primary_domain = row['primary_domain']
    page_url = row['page_url']

    sep.update_domain_info(page_url,domain_tags, primary_domain, collection_to_update)

5f1bb163896f82cdbda6100b'), 'title': 'Critical Theory', 'domain_tags': 'Social and Political Philosophy,Continental Philosophy', 'primary_domain': 'Continental Philosophy'}]
acknowledged: /entries/risk/
 True
[{'_id': ObjectId('5f1bb851896f82cdbda61432'), 'title': 'Risk', 'domain_tags': 'Ethics,Epistemology, Philosophy of Science, Philosophy of Economics', 'primary_domain': 'Ethics'}]
acknowledged: /entries/broad/
 True
[{'_id': ObjectId('5f1bb038896f82cdbda60f6d'), 'title': 'Charlie Dunbar Broad', 'domain_tags': 'Thinker, Philosophy of Science', 'primary_domain': 'Thinker'}]
acknowledged: /entries/cambridge-platonists/
 True
[{'_id': ObjectId('5f1bb05b896f82cdbda60f7b'), 'title': 'The Cambridge Platonists', 'domain_tags': 'Metaphysics', 'primary_domain': 'Metaphysics'}]
acknowledged: /entries/comparphil-chiwes/
 True
[{'_id': ObjectId('5f1bb0d8896f82cdbda60fc1'), 'title': 'Comparative Philosophy: Chinese and Western', 'domain_tags': 'Chinese Philosophy', 'primary_domain': 'Chinese Phi